**Facial Expression Recognition**

Classify facial expressions (e.g., happy vs. neutral) from imbalanced face image datasets.

Apply augmentation (flip, brightness), preprocess with edge enhancement, extract low-level and deep features (VGG16), perform serial fusion and PCA, and classify with Linear SVM.

Report accuracy, precision, recall, F1-score, and confusion matrix with 10-fold cross-validation

# Task: Binary Imbalanced Classification Problem
**Facial Expression Recognition**
Steps:

1. Data Preparation
    
  *  Data Augmentation (e.g., Flip and Rotate)

  *   Data Preprocessing (e.g., image enhancement)

2. Feature Engineering

   *   Low-level Features (HOG, LBP, GLCM)

   *   High-level / Deep Features (FC7 Layer of VGG19)

3. Feature Fusion and Dimensionality Reduction

    *  Feature Fusion (Serial-based) and Dimensionality Reduction (PCA)       

4. Classification (Linear SVM)


Note:

Cross-validation (K-fold) where k=10, Evaluation Metrics (Accuracy,
Precision, Recall, and F1-score). Also, show a confusion matrix.


Tools & Technologies

*  Python (OpenCV,scikit-learn, TensorFlow/Keras for VGG19).

*  Libraries: NumPy,Pandas, Matplotlib/Seaborn for visualization, etc.

**Extract Req. Data from Raw data**

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/fer2013.csv')

# Filter rows for happy (emotion=3) and neutral (emotion=6)
happy_neutral_df = df[df['emotion'].isin([3, 6])]

# Map labels: 3 -> 1 (happy), 6 -> 0 (neutral)
happy_neutral_df['emotion'] = happy_neutral_df['emotion'].replace({3: 1, 6: 0})

# Save to a new CSV file
happy_neutral_df.to_csv('happy_neutral_fer2013.csv', index=False)

<ipython-input-1-3d14dd0bee09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  happy_neutral_df['emotion'] = happy_neutral_df['emotion'].replace({3: 1, 6: 0})


In [ ]:
import pandas as pd

# Load the new CSV file
df = pd.read_csv('happy_neutral_fer2013.csv')

# Count the number of happy (emotion=1) and neutral (emotion=0) samples
happy_count = len(df[df['emotion'] == 1])
neutral_count = len(df[df['emotion'] == 0])

# Print the counts
print(f"Number of Happy samples: {happy_count}")
print(f"Number of Neutral samples: {neutral_count}")
print(f"Total samples: {happy_count + neutral_count}")

Number of Happy samples: 8989
Number of Neutral samples: 6198
Total samples: 15187


Oversampling of Neutral Images

In [ ]:
import pandas as pd
import numpy as np
from skimage.transform import rotate
from skimage.exposure import adjust_gamma

# ======================== Load Dataset and Inspect =============================
# Original counts:
# Happy samples: 8989
# Neutral samples: 6198
# Total: 15187

df = pd.read_csv('happy_neutral_fer2013.csv')

# Separate happy (majority) and neutral (minority)
happy_df = df[df['emotion'] == 1].copy()
neutral_df = df[df['emotion'] == 0].copy()

print("Original Dataset:")
print(f"Happy count: {len(happy_df)}")
print(f"Neutral count: {len(neutral_df)}")

# ======================== Step 1: Oversample Neutral Class ======================
# Target: Increase Neutral from 6198 to 8900 => Generate 2702 images

target_neutral = 8900
to_generate = target_neutral - len(neutral_df)

# Helper functions to convert between pixel string and image
def pixels_to_image(pixel_string):
    return np.array(pixel_string.split(), dtype=np.float32).reshape(48, 48)

def image_to_pixels(image):
    return ' '.join(map(str, image.flatten().astype(int)))

# Augmentation function
def augment_image(image, aug_type):
    if aug_type == 'flip':
        return np.fliplr(image)  # Horizontal flip
    elif aug_type == 'brighten':
        return adjust_gamma(image, gamma=1.2)  # Slight brightness increase
    elif aug_type == 'darken':
        return adjust_gamma(image, gamma=0.8)  # Slight brightness decrease
    elif aug_type == 'rotate':
        return rotate(image, angle=10, mode='edge')  # Slight rotation
    return image

np.random.seed(42)  # For reproducibility
augmented_rows = []
aug_types = ['flip', 'brighten', 'darken', 'rotate']
images_per_type = to_generate // len(aug_types)  # ~675 per type
remaining = to_generate % len(aug_types)

# Shuffle indices to avoid repetitive patterns
neutral_indices = np.random.permutation(len(neutral_df))

generated = 0
idx = 0

while generated < to_generate:
    row = neutral_df.iloc[neutral_indices[idx % len(neutral_df)]]
    image = pixels_to_image(row['pixels'])

    aug_type_idx = min(generated // images_per_type, len(aug_types) - 1)
    aug_type = aug_types[aug_type_idx]

    aug_image = augment_image(image, aug_type)
    aug_pixels = image_to_pixels(aug_image)

    new_row = row.copy()
    new_row['pixels'] = aug_pixels
    augmented_rows.append(new_row)

    generated += 1
    idx += 1

# Convert list of augmented rows into DataFrame
augmented_df = pd.DataFrame(augmented_rows)

# Combine original and augmented neutral data
new_neutral_df = pd.concat([neutral_df, augmented_df], ignore_index=True)

# Combine oversampled neutral with original happy (still imbalanced at this stage)
combined_oversampled_df = pd.concat([happy_df, new_neutral_df], ignore_index=True)

print("\nAfter Oversampling Neutral Class:")
print(f"Happy count: {len(happy_df)}")
print(f"Neutral count: {len(new_neutral_df)}")

# ======================== Step 2: Downsample Happy Class =======================
# Now downsample happy class to match neutral count (8900)

# Re-separate after oversampling
happy_df = combined_oversampled_df[combined_oversampled_df['emotion'] == 1].copy()
neutral_df = combined_oversampled_df[combined_oversampled_df['emotion'] == 0].copy()

# Downsample happy class to 8900 samples
np.random.seed(42)
happy_indices = np.random.choice(len(happy_df), size=8900, replace=False)
downsampled_happy_df = happy_df.iloc[happy_indices].reset_index(drop=True)

# Combine downsampled happy with oversampled neutral
final_balanced_df = pd.concat([downsampled_happy_df, neutral_df], ignore_index=True)

# Save final balanced dataset
final_balanced_df.to_csv('happy_neutral_balanced.csv', index=False)

# Print final balanced counts
print("\nFinal Balanced Dataset:")
print(f"Happy count: {len(final_balanced_df[final_balanced_df['emotion'] == 1])}")
print(f"Neutral count: {len(final_balanced_df[final_balanced_df['emotion'] == 0])}")

Original Dataset:
Happy count: 8989
Neutral count: 6198

After Oversampling Neutral Class:
Happy count: 8989
Neutral count: 8900

Final Balanced Dataset:
Happy count: 8900
Neutral count: 8900


In [ ]:
import pandas as pd
import numpy as np
import cv2

# Load the dataset
df = pd.read_csv('happy_neutral_balanced.csv')

# Function to convert pixel string to 48x48 image
def pixels_to_image(pixel_string):
    # Convert pixel string to array, handle out-of-range values
    pixels = np.array(pixel_string.split(), dtype=np.float32)
    # Clamp values to [0, 255] to prevent overflow
    pixels = np.clip(pixels, 0, 255)
    return pixels.reshape(48, 48).astype(np.uint8)

# Function to convert image back to pixel string
def image_to_pixels(image):
    return ' '.join(map(str, image.flatten().astype(int)))

# Preprocessing function
def preprocess_image(image):
    # Histogram Equalization for contrast enhancement
    image = cv2.equalizeHist(image)
    # Normalization to [0,1]
    image = image / 255.0
    # Scale back to [0,255] and clamp to prevent overflow
    image = np.clip(image * 255, 0, 255).astype(np.uint8)
    return image

# Apply preprocessing to all images
preprocessed_pixels = []
for pixel_string in df['pixels']:
    # Convert pixel string to image
    image = pixels_to_image(pixel_string)
    # Apply preprocessing
    image = preprocess_image(image)
    # Convert back to pixel string
    pixel_string = image_to_pixels(image)
    preprocessed_pixels.append(pixel_string)

# Update the DataFrame with preprocessed pixels
df['pixels'] = preprocessed_pixels

# Save to a new CSV file
df.to_csv('happy_neutral_preprocessed.csv', index=False)

# Print confirmation
print("Preprocessing complete. Preprocessed data saved to 'happy_neutral_preprocessed.csv'.")
print(f"Total samples: {len(df)}")

Preprocessing complete. Preprocessed data saved to 'happy_neutral_preprocessed.csv'.
Total samples: 17800


In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
from skimage.feature import hog, local_binary_pattern, graycomatrix, graycoprops
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import cv2

# Load the dataset
df = pd.read_csv('happy_neutral_preprocessed.csv')

# Function to convert pixel string to 48x48 image
def pixels_to_image(pixel_string):
    pixels = np.array(pixel_string.split(), dtype=np.uint8)
    return pixels.reshape(48, 48)

# Function to resize image for VGG19
def resize_for_vgg(image):
    return cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

# Extract Low-level Features
hog_features = []
lbp_features = []
glcm_features = []

for pixel_string in df['pixels']:
    # Convert to image
    image = pixels_to_image(pixel_string)

    # HOG Features
    hog_feature, _ = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    hog_features.append(hog_feature)

    # LBP Features
    lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    lbp_features.append(hist / hist.sum())  # Normalize

    # GLCM Features
    glcm = graycomatrix(image, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    glcm_props = [graycoprops(glcm, prop).ravel()[0] for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']]
    glcm_features.append(glcm_props)

# Convert feature lists to arrays
hog_features = np.array(hog_features)
lbp_features = np.array(lbp_features)
glcm_features = np.array(glcm_features)

# Extract High-level Features (VGG19 FC7 Layer)
base_model = VGG19(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)  # FC7 layer

vgg_features = []
for pixel_string in df['pixels']:
    image = pixels_to_image(pixel_string)
    image = resize_for_vgg(image)
    image = np.stack([image] * 3, axis=-1)  # Convert grayscale to RGB by stacking
    image = preprocess_input(image * 255)  # VGG19 expects [0,255] range
    image = np.expand_dims(image, axis=0)
    feature = model.predict(image, verbose=0)
    vgg_features.append(feature.flatten())

vgg_features = np.array(vgg_features)

# Add features to DataFrame
df['hog_features'] = [','.join(map(str, f)) for f in hog_features]
df['lbp_features'] = [','.join(map(str, f)) for f in lbp_features]
df['glcm_features'] = [','.join(map(str, f)) for f in glcm_features]
df['vgg19_features'] = [','.join(map(str, f)) for f in vgg_features]

# Save to a new CSV file
df.to_csv('happy_neutral_features.csv', index=False)

# Print confirmation
print("Feature extraction complete. Features saved to 'happy_neutral_features.csv'.")
print(f"Total samples: {len(df)}")

Feature extraction complete. Features saved to 'happy_neutral_features.csv'.
Total samples: 17800


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Step 1: Load the feature CSV
df = pd.read_csv('happy_neutral_features.csv')

# Step 2: Convert string features into numerical arrays
def parse_feature_column(column):
    return np.array([np.fromstring(f, sep=',') for f in df[column]])

# Step 3: Parse available features
hog = parse_feature_column('hog_features')
lbp = parse_feature_column('lbp_features')
glcm = parse_feature_column('glcm_features')

# Step 4: Check if VGG features exist
if 'vgg_features' in df.columns:
    vgg = parse_feature_column('vgg_features')
    fused_features = np.hstack([hog, lbp, glcm, vgg])
    print("VGG features included in fusion.")
else:
    fused_features = np.hstack([hog, lbp, glcm])
    print("Warning: 'vgg_features' column not found. Proceeding without VGG features.")

# Step 5: Feature Scaling
scaler = StandardScaler()
fused_scaled = scaler.fit_transform(fused_features)

# Step 6: PCA Dimensionality Reduction
pca = PCA(n_components=0.99, random_state=42)  # retain 99% variance
fused_pca = pca.fit_transform(fused_scaled)

# Step 7: Output
print("Original feature shape:", fused_features.shape)
print("Reduced feature shape after PCA:", fused_pca.shape)

# Optional: Save for classification step
np.save('fused_features_pca.npy', fused_pca)
np.save('labels.npy', df['emotion'].values)


Original feature shape: (17800, 914)
Reduced feature shape after PCA: (17800, 408)


In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Step 1: Load features and labels
X = np.load('fused_features_pca.npy')  # shape: (samples, reduced_features)
y = np.load('labels.npy')              # shape: (samples,)

# Step 2: Define 10-fold stratified cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Step 3: Initialize Linear SVM
svm = LinearSVC(max_iter=10000, random_state=42)

# Step 4: Cross-validate and predict
print("Performing 10-Fold Cross-Validation...")
y_pred = cross_val_predict(svm, X, y, cv=cv)

# Step 5 (Alternative): Get numerical values of metrics
precision, recall, f1, support = precision_recall_fscore_support(y, y_pred, average=None, labels=[0, 1])
accuracy = accuracy_score(y, y_pred)

print("Class-wise Metrics:")
print(f"Neutral - Precision: {precision[0]:.2f}, Recall: {recall[0]:.2f}, F1: {f1[0]:.2f}")
print(f"Happy   - Precision: {precision[1]:.2f}, Recall: {recall[1]:.2f}, F1: {f1[1]:.2f}")
print(f"\nOverall Accuracy: {accuracy:.2f}")


Performing 10-Fold Cross-Validation...
Class-wise Metrics:
Neutral - Precision: 0.80, Recall: 0.79, F1: 0.80
Happy   - Precision: 0.80, Recall: 0.81, F1: 0.80

Overall Accuracy: 0.80


In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from joblib import dump  # or use pickle if you prefer

# Load fused PCA features and labels
X = np.load('fused_features_pca.npy')
y = np.load('labels.npy')

# Initialize and train Linear SVM on full data
svm = LinearSVC(max_iter=10000, random_state=42)
svm.fit(X, y)

# Save the model
dump(svm, 'linear_svm_model.joblib')
dump(scaler, 'scaler.joblib')
dump(pca, 'pca.joblib')

print("Model saved as 'linear_svm_model.joblib'")

Model saved as 'linear_svm_model.joblib'


In [ ]:
# predict_emotion.py
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern, graycomatrix, graycoprops
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from joblib import load

# Load the trained model, scaler, and PCA
try:
    svm = load('linear_svm_model.joblib')
    scaler = load('scaler.joblib')
    pca = load('pca.joblib')
except FileNotFoundError as e:
    print(f"Error: {e}. Ensure 'linear_svm_model.joblib', 'scaler.joblib', and 'pca.joblib' are in the directory.")
    exit()

# Preprocess the image (same as training)
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Could not load image from {image_path}")
    image = cv2.resize(image, (48, 48), interpolation=cv2.INTER_AREA)
    image = cv2.equalizeHist(image)
    return image

# Feature extraction (adjusted to match training)
def extract_features(image):
    # HOG (adjust parameters to match ~324 features)
    hog_f, _ = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    print(f"HOG shape: {hog_f.shape}")  # Debug
    # If HOG is still 900, try reducing cell size or block overlap
    if hog_f.shape[0] != 324:  # Expected training HOG size
        hog_f, _ = hog(image, orientations=9, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=True)
        print(f"Adjusted HOG shape: {hog_f.shape}")  # Debug
    # LBP
    lbp = local_binary_pattern(image, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    lbp_f = hist / hist.sum()
    print(f"LBP shape: {lbp_f.shape}")  # Debug
    # GLCM
    glcm = graycomatrix(image, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    glcm_f = [graycoprops(glcm, prop).ravel()[0] for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']]
    glcm_f = np.array(glcm_f)
    print(f"GLCM shape: {glcm_f.shape}")  # Debug
    # VGG16 FC1
    base_model = VGG16(weights='imagenet', include_top=True)
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)  # FC1: 4096
    img = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
    img = np.stack([img] * 3, axis=-1)  # Convert to RGB
    img = preprocess_input(img * 255)
    img = np.expand_dims(img, axis=0)
    vgg_f = model.predict(img, verbose=0).flatten()
    print(f"VGG16 shape: {vgg_f.shape}")  # Debug
    # Fuse features
    fused = np.hstack([hog_f, lbp_f, glcm_f, vgg_f])
    print(f"Fused shape before adjustment: {fused.shape}")  # Debug
    # Adjust to match training's 914 features
    expected_total = 914
    hog_len, lbp_len, glcm_len = len(hog_f), len(lbp_f), len(glcm_f)
    vgg_len = expected_total - (hog_len + lbp_len + glcm_len)
    if vgg_len <= 0 or vgg_len > len(vgg_f):
        raise ValueError(f"Cannot adjust VGG16 features to fit {expected_total}. Got VGG length {len(vgg_f)}, need {vgg_len}. Check HOG parameters.")
    fused = np.hstack([hog_f, lbp_f, glcm_f, vgg_f[:vgg_len]])
    print(f"Fused shape after adjustment: {fused.shape}")  # Debug
    # Apply scaler and PCA
    if fused.shape[0] != expected_total:
        raise ValueError(f"Expected {expected_total} features, got {fused.shape[0]}. Adjust feature extraction.")
    fused_scaled = scaler.transform(fused.reshape(1, -1))
    fused_pca = pca.transform(fused_scaled)
    print(f"PCA shape: {fused_pca.shape}")  # Debug
    return fused_pca

# Predict
def predict_emotion(image_path):
    image = preprocess_image(image_path)
    features = extract_features(image)
    prediction = svm.predict(features)
    return "Happy" if prediction[0] == 1 else "Neutral"

# Main execution
if __name__ == "__main__":
    image_path = "/content/gettyimages-1465122813-612x612.jpg"  # Replace with your image path
    try:
        result = predict_emotion(image_path)
        print(f"Prediction: {result}")
    except Exception as e:
        print(f"Error: {e}")

HOG shape: (900,)
Adjusted HOG shape: (144,)
LBP shape: (9,)
GLCM shape: (5,)
VGG16 shape: (4096,)
Fused shape before adjustment: (4254,)
Fused shape after adjustment: (914,)
PCA shape: (1, 408)
Prediction: Happy
